## 휴먼지능정보공학과 201910787 김영서

In [1]:
import sys
import os

home=os.path.expanduser("~")
os.environ["PYSPARK_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.9-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

## 문제 1: 열린데이터 수집

서울시 열린데이터에서 검색을 하면 '서울시 우리마을가게 상권분석서비스(상권-아파트)'가 있다.
(https://data.seoul.go.kr/dataList/OA-15566/S/1/datasetView.do?tab=A)
OpenAPI를 통해서 2021년 최근 100개 데이터를 JSON 형식으로 수집하자.

### 문제 1-1의 답
*  프로그램으로 url을 만들어 출력. 그 url을 클릭하면 데이터가 출력 되어야 한다.

In [3]:
%%writefile src/key.properties
dataseoul=636347656d7a6171353074504d575a

Overwriting src/key.properties


In [8]:
import os
import src.mylib

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=src.mylib.getKey(keyPath)
KEY=str(key['dataseoul'])

In [12]:
# api
TYPE='json'
SERVICE='InfoTrdarAptQq'
START_INDEX=str(1)
END_INDEX=str(10)

In [14]:
params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX])
print(params)

636347656d7a6171353074504d575a/json/InfoTrdarAptQq/1/10


In [15]:
import urllib
_url='http://openAPI.seoul.go.kr:8088/'
url=urllib.parse.urljoin(_url,params)
print(url) # backslashes removed

http://openAPI.seoul.go.kr:8088/636347656d7a6171353074504d575a/json/InfoTrdarAptQq/1/10


### 문제 1-2의 답
* 전체 데이터 중 1건만 출력.

In [44]:
import requests

_data=requests.get(url)

In [45]:
data=_data.json()

In [49]:
data['InfoTrdarAptQq']['row'][0]

{'STDR_YY_CD': '2021',
 'TRDAR_SE_CD': 'D',
 'TRDAR_SE_CD_NM': '발달상권',
 'APT_HSMP_CO': 3.0,
 'AE_66_SQMT_BELO_HSHLD_CO': 157.0,
 'AE_66_SQMT_HSHLD_CO': 0.0,
 'AE_99_SQMT_HSHLD_CO': 0.0,
 'AE_132_SQMT_HSHLD_CO': 0.0,
 'AE_165_SQMT_HSHLD_CO': 0.0,
 'PC_1_HDMIL_BELO_HSHLD_CO': 0.0,
 'PC_1_HDMIL_HSHLD_CO': 157.0,
 'PC_2_HDMIL_HSHLD_CO': 0.0,
 'PC_3_HDMIL_HSHLD_CO': 0.0,
 'PC_4_HDMIL_HSHLD_CO': 0.0,
 'PC_5_HDMIL_HSHLD_CO': 0.0,
 'PC_6_HDMIL_ABOVE_HSHLD_CO': 0.0,
 'AVRG_AE': 31.0,
 'AVRG_MKTC': 161157476.0,
 'STDR_QU_CD': '1',
 'TRDAR_CD': '1001158',
 'TRDAR_CD_NM': '당산역_2'}

### 문제 1-3의 답
* 다음 항목을 100개 정렬해서 출력.

    TRDAR_SE_CD_NM	상권_구분_코드_명<br>
	APT_HSMP_CO	아파트_단지_수<br>
	PC_6_HDMIL_ABOVE_HSHLD_CO	아파트_가격_6_억_이상_세대_수<br>
	AVRG_MKTC	아파트_평균_시가<br>
	TRDAR_CD_NM	상권_코드_명

In [17]:
%%writefile src/InfoTrdarAptQq_json.py
#!/usr/bin/env python
# coding: utf-8
import os
import requests
import urllib
import mylib # NO! src.mylib

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=str(key['dataseoul'])
    TYPE='json'
    #OLD: SERVICE='SearchSTNBySubwayLineService'
    SERVICE='InfoTrdarAptQq'
    LINE_NUM=str(2)
    START_INDEX=str(1)
    END_INDEX=str(10)
    startIndex=1
    endIndex=10
    list_total_count=0    # set later
    a = 1
    while True:
        START_INDEX=str(startIndex)
        END_INDEX=str(endIndex)
        #params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
        params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,'','',LINE_NUM])
        # (2) make a full url
        _url='http://openAPI.seoul.go.kr:8088' #NOTE slash: do not use 'http://openAPI.seoul.go.kr:8088/'
        #url=urllib.parse.urljoin(_url,params)
        url="/".join([_url,params])
        #print(url)
        # (3) get data
        r=requests.get(url)
        #print(r)
        data=r.json()
        #print(stations)
        if(startIndex==1):
            list_total_count=data['InfoTrdarAptQq']['list_total_count']
            print("- Total Count: ", list_total_count)
        for e in data['InfoTrdarAptQq']['row']:
            print (a,"{0:4}\t{1:4}\t{2:4}\t{3:3}\t{4:5}".format(e['TRDAR_SE_CD_NM'], e['APT_HSMP_CO'], e['PC_6_HDMIL_ABOVE_HSHLD_CO'], e['AVRG_MKTC'], e['TRDAR_CD_NM']))
            a+=1
        startIndex+=10
        endIndex+=10
        if(endIndex > 100):
            print("----- Ending endIndex=",endIndex)
            break  # exit from the while loop

if __name__ == "__main__":
    doIt()

Overwriting src/InfoTrdarAptQq_json.py


In [18]:
!python3 src/InfoTrdarAptQq_json.py

- Total Count:  38654
1 발달상권	 3.0	 0.0	161157476.0	당산역_2
2 골목상권	 5.0	 0.0	150247619.0	동남로82길
3 관광특구	 8.0	 0.0	114850602.0	종로·청계 관광특구
4 관광특구	11.0	291.0	245511812.0	동대문패션타운 관광특구
5 관광특구	70.0	105.0	428764829.0	이태원 관광특구
6 전통시장	39.0	 1.0	191078841.0	길동복조리시장
7 전통시장	 8.0	 0.0	197751693.0	둔촌역전통시장
8 전통시장	11.0	 0.0	152871984.0	성내골목시장
9 전통시장	 1.0	 0.0	176000000.0	동서울시장
10 전통시장	 2.0	 0.0	156144643.0	고분다리골목시장
11 전통시장	17.0	 0.0	187541526.0	양지골목시장
12 전통시장	15.0	 0.0	130506269.0	암사종합시장
13 전통시장	 1.0	 0.0	177666667.0	고덕전통시장
14 전통시장	 5.0	 0.0	238201494.0	명일골목시장
15 전통시장	47.0	 0.0	215808647.0	새마을시장
16 전통시장	14.0	60.0	242990364.0	석촌시장 
17 전통시장	178.0	19.0	214028232.0	방이시장 
18 전통시장	 6.0	 0.0	204376515.0	마천시장 
19 전통시장	19.0	 0.0	160941704.0	풍납시장 
20 전통시장	 2.0	76.0	759927632.0	도곡시장 
21 전통시장	 1.0	48.0	2708416667.0	청담삼익시장
22 전통시장	 3.0	 9.0	537022727.0	논현종합시장
23 전통시장	19.0	 2.0	234485808.0	영동전통시장
24 전통시장	 1.0	 7.0	400638298.0	강남시장_강남
25 전통시장	 2.0	 0.0	193867857.0	양재시장 
26 전통시장	 4.0	 0.0	313638333.0	남부종합시장
27 전통시장	 1.0	

## 문제 2: RDD를 사용해서 단어빈도 계산

In [20]:
%%writefile data/yesterday_lyrics.txt
Yesterday 어제는
All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데
Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여
Oh, I believe in yesterday 오, 어제가 좋았는데
Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021
I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어
There's a shadow hanging over me 내 위에 그림자가 드리워져 있네
Oh!, yesterday came suddenly 오!, 이제는 너무 갑자기 와버렸어..

Overwriting data/yesterday_lyrics.txt


In [181]:
myRdd=spark.sparkContext\
    .textFile(os.path.join("data","yesterday_lyrics.txt"))

In [48]:
myRdd.collect()

['Yesterday 어제는',
 'All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데',
 "Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여",
 'Oh, I believe in yesterday 오, 어제가 좋았는데',
 'Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021',
 "I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어",
 "There's a shadow hanging over me 내 위에 그림자가 드리워져 있네",
 'Oh!, yesterday came suddenly 오!, 이제는 너무 갑자기 와버렸어..']

### 문제 2-1의 답
* 단어빈도를 계산해서 출력

출력 예시: [('Yesterday', 1),('어제는', 1),('All', 1),('my', 1),...]

In [210]:
wc=myRdd\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .take(100)
for i in wc:
    print (i, end=',')

('Yesterday', 1),('어제는', 1),('All', 1),('my', 1),('troubles', 1),('seemed', 1),('to', 1),('far', 1),('away', 1),('내', 1),('모든', 1),('문제들이', 1),('참', 1),('멀리', 1),('있는', 1),('것', 1),('같았는데', 1),('Now', 1),('it', 1),('looks', 1),('as', 1),('though', 1),("they're", 1),('here', 1),('to', 1),('stay', 1),('이제는', 1),('그', 1),('모든', 1),('문제들이', 1),('여기에', 1),('있는', 1),('것처럼', 1),('보여', 1),('Oh,', 1),('I', 1),('believe', 1),('in', 1),('yesterday', 1),('오,', 1),('어제가', 1),('좋았는데', 1),('Suddenly', 1),('갑자기', 1),('그-림-자-가', 1),('2021', 1),('그-림-자-가', 1),('2021', 1),("I'm", 1),('not', 1),('half', 1),('the', 1),('man', 1),('I', 1),('used', 1),('to', 1),('be', 1),('난', 1),('예전', 1),('내', 1),('모습의', 1),('반도', 1),('못한', 1),('사람이', 1),('되어버렸어', 1),("There's", 1),('a', 1),('shadow', 1),('hanging', 1),('over', 1),('me', 1),('내', 1),('위에', 1),('그림자가', 1),('드리워져', 1),('있네', 1),('Oh!,', 1),('yesterday', 1),('came', 1),('suddenly', 1),('오!,', 1),('이제는', 1),('너무', 1),('갑자기', 1),('와버렸어..', 1),

### 문제 2-2의 답
* 빈도순으로 출력, 빈도 1이하는 제외.

In [41]:
wc=myRdd\
    .flatMap(lambda x:x.split())\
    .filter(lambda x: x.lower() not in stopwords)\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(100)

In [44]:
for i in wc:
    if i[0] != 1:
        print (i[0], i[1])

3 to
3 내
2 모든
2 문제들이
2 있는
2 이제는
2 I
2 yesterday
2 갑자기
2 그-림-자-가
2 2021


### 문제 2-3의 답
* 소문자, 하이픈, 느낌표, 숫자를 제외하고 단어빈도 계산해서 출력 (빈도 1이하는 제외)

* 소문자: 'Yesterday', 'yesterday'를 동일한 단어로 계산

* 하이픈: '그림자가', '그-림-자-가'를 동일한 단어로 계산

* 느낌표: 'oh', 'oh!'를 동일한 단어로 계산

* 숫자: '2021'는 제외하고 계산

In [206]:
wc2=myRdd.map(lambda x: [i.lower() for i in x])
print (wc2.collect())

[['y', 'e', 's', 't', 'e', 'r', 'd', 'a', 'y', ' ', '어', '제', '는'], ['a', 'l', 'l', ' ', 'm', 'y', ' ', 't', 'r', 'o', 'u', 'b', 'l', 'e', 's', ' ', 's', 'e', 'e', 'm', 'e', 'd', ' ', 't', 'o', ' ', 'f', 'a', 'r', ' ', 'a', 'w', 'a', 'y', ' ', '내', ' ', '모', '든', ' ', '문', '제', '들', '이', ' ', '참', ' ', '멀', '리', ' ', '있', '는', ' ', '것', ' ', '같', '았', '는', '데'], ['n', 'o', 'w', ' ', 'i', 't', ' ', 'l', 'o', 'o', 'k', 's', ' ', 'a', 's', ' ', 't', 'h', 'o', 'u', 'g', 'h', ' ', 't', 'h', 'e', 'y', "'", 'r', 'e', ' ', 'h', 'e', 'r', 'e', ' ', 't', 'o', ' ', 's', 't', 'a', 'y', ' ', '이', '제', '는', ' ', '그', ' ', '모', '든', ' ', '문', '제', '들', '이', ' ', '여', '기', '에', ' ', '있', '는', ' ', '것', '처', '럼', ' ', '보', '여'], ['o', 'h', ',', ' ', 'i', ' ', 'b', 'e', 'l', 'i', 'e', 'v', 'e', ' ', 'i', 'n', ' ', 'y', 'e', 's', 't', 'e', 'r', 'd', 'a', 'y', ' ', '오', ',', ' ', '어', '제', '가', ' ', '좋', '았', '는', '데'], ['s', 'u', 'd', 'd', 'e', 'n', 'l', 'y', ' ', '갑', '자', '기', ' ', '그', '-', '림', '-', 

In [208]:
stopwords = set(["-", "!","0", "1","2","3","4","5","6","7,","8,","9"])
wc2=myRdd\
    .flatMap(lambda x:x.split())\
    .filter(lambda x: x.lower() not in stopwords)\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(100)

In [209]:
for i in wc2:
    if i[0] != 1:
        print (i[0], i[1])

3 to
3 내
2 모든
2 문제들이
2 있는
2 이제는
2 I
2 yesterday
2 갑자기
2 그-림-자-가
2 2021


### 문제 2-4의 답
* 1글자 단어 (예를 들어 'i', '내') 제외하고 단어빈도 계산 (빈도 1이하는 제외)

In [205]:
stopwords = set(["i","내"])
wc2=myRdd\
    .flatMap(lambda x:x.split())\
    .filter(lambda x: x.lower() not in stopwords)\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(100)

In [143]:
for i in wc2:
    if i[0] != 1:
        print (i[0], i[1])

3 to
2 모든
2 문제들이
2 있는
2 이제는
2 yesterday
2 갑자기
2 그-림-자-가
2 2021


## 문제 3: DataFrame을 사용해서 코로나 발생동향 분석

서울시 열린데이터에서 검색을 하면 '서울특별시 코로나19 자치구별 확진자 발생동향'이 있다.
https://data.seoul.go.kr/dataList/OA-20470/S/1/datasetView.do
데이터를 '내려받기'해서 '서울특별시\ 코로나19\ 자치구별\ 확진자\ 발생동향.json' 파일을 분석하세요.

### 문제 3-1의 답
* 전체 건수 출력. null 값을 제외한 null 건수 출력 (null이 남아있는지 확인하기 위해)

In [108]:
import os
_jfname=os.path.join('data','서울특별시 코로나19 자치구별 확진자 발생동향.json')
with open(_jfname, 'rb') as f:
    data = f.readlines()

In [104]:
jfile= os.path.join('data','서울특별시 코로나19 자치구별 확진자 발생동향.json')
covidDf= spark.read.json(jfile)

In [145]:
len(data)

633

### 문제 3-2의 답
*  (null제거하고) 종로, 마포, 용산, 합계 (종로, 마포, 용산) 10건 출력.

|      |    |       |      |
|------|----|-------|------|
|jongno|mapo|yongsan|   tot|
|  1968|4127|   3031|9126.0|

### 문제 3-3의 답
*  (null제거하고) 종로구의 2020년 3월 데이터 출력.

|       jcg_dt|jongno|
|-------------|------|
|2020.03.31.10|    13|


### 문제 3-4의 답
* (null제거하고) 년 x 월별 합계 10건 출력.

|     |       |       |
|-----|-------|-------|
|month|   2020|   2021|
|   07| 1273.0|62849.0|